In [ ]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

In [ ]:
# Set up constant
kBT_mu = 0.01 # Boltzmann constant measured interms of chemical potential

# Set up the variable space
En_mu = np.linspace(0, 6, 200) #Energy measured in units of chemical potential
f = 1.0/(np.exp( (En_mu-1)/kBT_mu )+1) # Occupation number according to F-D statistics

In [ ]:
# Set up plot
plot = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 6], y_range=[0.0, 1.1])

r = plot.line(En_mu, f,line_width=3, line_alpha=0.6)

In [ ]:
# Set up callbacks to update the live graph
def update_data(a):
    # Generate the new curve
    En_mu = np.linspace(0, 6, 200) #Energy measured in units of chemical potential
    r.data_source.data['y'] = 1.0/(np.exp( (En_mu-1)/a )+1)
    push_notebook()

In [ ]:
show(plot, notebook_handle = True)

In [ ]:
interact(update_data,a=(0.01,1,0.01))